# Processamento de linguagem natural com Spacy

In [ ]:
# instalando as bibliotecas necessárias
!spacy
!pandas
!ipython
!matplotlib

In [ ]:
# importando a biblioteca spaCy
import spacy
from spacy.cli import download
from spacy import displacy
import re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# baixando o modelo de linguagem em português
#download("pt_core_news_sm")
download("pt_core_news_lg")

In [ ]:
# carregando o modelo de linguagem em português
nlp = spacy.load("pt_core_news_lg")


## Preparando o documento

In [ ]:
# lendo o arquivo de texto
texto = "./helena.txt"


In [ ]:
# abrindo o arquivo e lendo o conteúdo
with open(texto, "r", encoding="utf-8") as file:
    texto = file.read()


### Limpeza simples do texto

In [ ]:
#limpar o texto (remover espaços extras, quebras de linha desnecessárias, colocar em minusculas, remover pontuações)
texto_limpo = re.sub(r'\s+', ' ', texto)  # remover quebras de linha e espaços extras
texto_limpo = texto_limpo.strip()  # remover espaços no início e no fim
texto_limpo = texto_limpo.lower()  # converter para minúsculas

## Processar o texto com o modelo

In [ ]:
# processando o texto com o modelo carregado
doc = nlp(texto_limpo)

### Token

O token é a unidade básica de texto em processamento de linguagem natural (PLN). Pode ser uma palavra, um número, um símbolo ou até mesmo um espaço em branco. A tokenização é o processo de dividir um texto em tokens, facilitando a análise e o processamento do conteúdo textual.

### Partes do Discurso (POS - Part of Speech)

As partes do discurso (POS) são categorias gramaticais que descrevem a função de uma palavra em uma frase. Exemplos comuns de partes do discurso incluem substantivos, verbos, adjetivos, advérbios, pronomes, preposições, conjunções e interjeições. A identificação correta das partes do discurso é essencial para entender o significado e a estrutura das frases.

### Dependências

As dependências gramaticais descrevem as relações entre as palavras em uma frase. Elas indicam como as palavras estão conectadas umas às outras, mostrando quais palavras dependem de outras para formar um significado completo. Por exemplo, em uma frase como "O gato está no telhado", a palavra "gato" depende do verbo "está" para completar o sentido da frase.

In [ ]:
# listar primeiras 10 tokens do texto com suas respectivas partes do discurso e dependências
print("Tokens do texto (de 1000 a 1040), com POS e DEP:")
for token in doc[1000:1040]: # iterando sobre os tokens do texto
    print(token.text, token.pos_, token.dep_)


### Tabela de POS

| Abreviação | Parte do Discurso       | Descrição                                      |
|------------|------------------------|------------------------------------------------|
| NOUN       | Substantivo            | Nome de pessoas, lugares, coisas ou ideias     |
| VERB       | Verbo                  | Ação ou estado de ser                           |
| ADP        | Adposição              | Inclui preposições e pós-posições               |
| ADJ        | Adjetivo               | Descreve ou modifica um substantivo               |
| ADV        | Advérbio               | Modifica um verbo, adjetivo ou outro advérbio     |
| PRON       | Pronome                | Substitui um substantivo                          |
| PREP       | Preposição             | Indica relações entre palavras                   |
| CONJ       | Conjunção              | Conecta palavras, frases ou orações          |
| INTJ       | Interjeição            | Expressa emoção ou reação                        |
| DET        | Determinante           | Especifica um substantivo                         |
| AUX        | Verbo Auxiliar         | Ajuda a formar tempos verbais, modos ou vozes    |
| PUNCT      | Pontuação              | Símbolos de pontuação                             |



### Tabela de dependências

| Abreviação | Descrição                                      |
|------------|------------------------------------------------|
| nsubj      | Sujeito nominal                                |
| obj        | Objeto direto                                  |
| iobj       | Objeto indireto                                |
| amod       | Modificador adjetival                          |
| advmod     | Modificador adverbial                          |
| nmod       | Modificador nominal                            |
| det        | Determinante                                   |
| case       | Palavra de caso (preposição)                   |
| root       | Raiz da frase                                  |

### Lemmatizar

O processo de lematização consiste em reduzir uma palavra à sua forma base ou dicionário, conhecida como lema. Por exemplo, as palavras "correr", "correndo" e "correu" têm o mesmo lema "correr". A lematização é útil em várias aplicações de processamento de linguagem natural (PLN), como análise de sentimentos, recuperação de informações e tradução automática, pois ajuda a agrupar diferentes formas de uma palavra sob um único conceito.

In [ ]:
# lemmas dos 50 primeiros verbos
print("\nLemas dos primeiros 50 verbos:")
verb_count = 0 # contador de verbos encontrados
for token in doc:
    if token.pos_ == "VERB": # verificando se o token é um verbo
        print(token.text, token.lemma_)
        verb_count += 1
    if verb_count >= 50: # se já encontrou 100 verbos, sai do loop
        break

In [ ]:
# contar os 10 verbos mais comuns no texto (após lematização)
from collections import Counter # importando Counter para contagem
verb_lemmas = [token.lemma_ for token in doc if token.pos_ == "VERB"] # lista de lemas dos verbos usando list comprehension
verb_freq = Counter(verb_lemmas) # contando a frequência dos lemas dos verbos
print("\n10 verbos mais comuns no texto (após lematização):")
for verb, freq in verb_freq.most_common(10):
    print(verb, freq)


### NER

A Reconhecimento de Entidades Nomeadas (NER - Named Entity Recognition) é uma técnica de processamento de linguagem natural (PLN) que identifica e classifica entidades mencionadas em um texto em categorias predefinidas, como pessoas, organizações, locais, datas, valores monetários, entre outras. O NER é amplamente utilizado em várias aplicações, como análise de sentimentos, extração de informações e sistemas de perguntas e respostas.

#### Tabela de NER

| Abreviação | Entidade               | Descrição                                      |
|------------|-----------------------|------------------------------------------------|
| PERSON     | Pessoa                | Nomes de pessoas, personagens ou indivíduos     |
| ORG        | Organização           | Nomes de empresas, instituições ou grupos       |
| GPE        | Localização           | Nomes de países, cidades ou estados             |
| LOC        | Local                 | Nomes de locais geográficos, como montanhas ou rios |
| DATE       | Data                  | Referências a datas ou períodos de tempo        |    
| TIME       | Hora                  | Referências a horários ou períodos do dia       |
| MISC       | Miscelânea            | Entidades que não se encaixam em outras categorias |

In [ ]:
# listar 10 Pessoas mais comuns no texto
person_entities = [ent.text for ent in doc.ents if ent.label_ == "PER"] # lista de entidades do tipo Pessoa
person_freq = Counter(person_entities) # contando a frequência das entidades Pessoa
print("\n10 Pessoas mais comuns no texto:")
for person, freq in person_freq.most_common(10):
    print(person, freq)

In [ ]:
# listar 10 localizações mais comuns no texto
location_entities = [ent.text for ent in doc.ents if ent.label_ == "LOC"] # lista de entidades do tipo Localização
location_freq = Counter(location_entities) # contando a frequência das entidades Localização
print("\n10 Localizações mais comuns no texto:")
for location, freq in location_freq.most_common(10):
    print(location, freq)

#### Visualizar NER com displacy

O displacy é uma ferramenta de visualização integrada ao spaCy que permite exibir entidades nomeadas (NER) e dependências gramaticais de forma interativa e visualmente atraente. Ele pode ser usado em notebooks Jupyter ou em páginas web para facilitar a análise e compreensão do texto processado.

In [ ]:
# usar o displacy para visualizar as entidades nomeadas
html = displacy.render(doc, style="ent", jupyter=False)
#salvar em um arquivo HTML
with open("entidades.html", "w", encoding="utf-8") as file:
    file.write(html)


## Análise semântica com vetores de palavras

Com spacy é possível acessar os vetores de palavras (word vectors) que representam semanticamente as palavras em um espaço vetorial. Esses vetores capturam relações semânticas entre palavras, permitindo realizar operações como encontrar palavras semelhantes, calcular similaridade entre palavras e frases, e realizar tarefas de clustering e classificação baseadas em significado.

Vamos testar com as palavras "mulher", "homem", "escravo", "senhor", "negro", "branco". O objetivo é verificar quais palavras no texto são semanticamente mais próximas dessas três palavras.


In [ ]:
# vamos explorar os vetores de palavras mulher e escravo
word1 = nlp("mulher")

# encontrar as 10 palavras mais similares a "mulher"
print("\n10 palavras mais similares a 'mulher':")
similar_words = sorted(
    doc.vocab,
    key=lambda w: word1.similarity(nlp(w.text)), # calculando similaridade
    reverse=True
    ) # ordenando palavras por similaridade
count = 0
for w in similar_words:
    if w.has_vector and w.is_lower and w.is_alpha:# garantindo que a palavra tem vetor, é minúscula e é alfabética
        print(w.text, word1.similarity(nlp(w.text)))# imprimindo palavra e similaridade
        count += 1
    if count >= 10:
        break

In [ ]:
# agora para a palavra "homem"
word2 = nlp("homem")

# encontrar as 10 palavras mais similares a "homem"
print("\n10 palavras mais similares a 'homem':")
similar_words = sorted(doc.vocab, key=lambda w: word2.similarity(nlp(w.text)), reverse=True)
count = 0
for w in similar_words:
    if w.has_vector and w.is_lower and w.is_alpha:
        print(w.text, word2.similarity(nlp(w.text)))
        count += 1
    if count >= 10:
        break

In [ ]:
# "escravo"
word3 = nlp("escravo")
# encontrar as 10 palavras mais similares a "escravo"
print("\n10 palavras mais similares a 'escravo':")
similar_words = sorted(doc.vocab, key=lambda w: word3.similarity(nlp(w.text)), reverse=True)
count = 0
for w in similar_words:
    if w.has_vector and w.is_lower and w.is_alpha:
        print(w.text, word3.similarity(nlp(w.text)))
        count += 1
    if count >= 10:
        break                                                                    

In [ ]:
# "senhor"
word4 = nlp("senhor")
# encontrar as 10 palavras mais similares a "negro"
print("\n10 palavras mais similares a 'senhor':")
similar_words = sorted(doc.vocab, key=lambda w: word4.similarity(nlp(w.text)), reverse=True)
count = 0
for w in similar_words:
    if w.has_vector and w.is_lower and w.is_alpha:
        print(w.text, word4.similarity(nlp(w.text)))
        count += 1
    if count >= 10:
        break   

In [ ]:
# "negro"
word5 = nlp("negro")
# encontrar as 10 palavras mais similares a "negro"
print("\n10 palavras mais similares a 'negro':")
similar_words = sorted(doc.vocab, key=lambda w: word4.similarity(nlp(w.text)), reverse=True)
count = 0
for w in similar_words:
    if w.has_vector and w.is_lower and w.is_alpha:
        print(w.text, word5.similarity(nlp(w.text)))
        count += 1
    if count >= 10:
        break                                                                    

In [ ]:
# "branco"
word6 = nlp("branco")
# encontrar as 10 palavras mais similares a "branco"
print("\n10 palavras mais similares a 'branco':")
similar_words = sorted(doc.vocab, key=lambda w: word5.similarity(nlp(w.text)), reverse=True)
count = 0
for w in similar_words:
    if w.has_vector and w.is_lower and w.is_alpha:
        print(w.text, word6.similarity(nlp(w.text)))
        count += 1
    if count >= 10:
        break  

In [ ]:
!pip install matplotlib

## Visualização de similaridade semântica com gráfico 2D

In [ ]:
def top_n_similar_from_vocab(target_vec, target_text, vocab, n=7):
    """Retorna as n palavras mais similares ao vetor alvo a partir do vocabulário fornecido."""
    candidates = []
    tnorm = np.linalg.norm(target_vec)
    for lex in vocab:
        if not (lex.has_vector and lex.is_lower and lex.is_alpha):
            continue
        if lex.text == target_text:
            continue
        v = lex.vector
        vn = np.linalg.norm(v)
        if vn == 0 or tnorm == 0:
            continue
        sim = float(np.dot(target_vec, v) / (tnorm * vn))
        candidates.append((lex.text, sim))
    candidates.sort(key=lambda x: x[1], reverse=True)
    return candidates[:n]

In [ ]:
# mulher (azul) e homem (verde)
target1_text = "mulher"
target2_text = "homem"
t1 = nlp(target1_text)[0]
t2 = nlp(target2_text)[0]

if nlp.vocab.vectors_length == 0:
    raise RuntimeError("O modelo atual não tem vetores (nlp.vocab.vectors_length == 0). "
                       "Instale um modelo com vetores: python -m spacy download pt_core_news_md (ou lg).")

topn = 10
top1 = [w for w,_ in top_n_similar_from_vocab(t1.vector, t1.text, nlp.vocab, n=topn)]
top2 = [w for w,_ in top_n_similar_from_vocab(t2.vector, t2.text, nlp.vocab, n=topn)]

# criar lista única preservando ordem (primeiro alvo1 e seus top, depois alvo2 e seus top não repetidos)
words = [target1_text] + top1 + [w for w in [target2_text] + top2 if w not in ([target1_text] + top1)]

# montar matriz de vetores
vecs = [nlp.vocab[w].vector for w in words]
X = np.vstack(vecs)

# projetar em 2D com SVD
Xc = X - X.mean(axis=0)
U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
proj = Xc.dot(Vt[:2].T)

# determinar cor por origem
colors = []
for w in words:
    if w == target1_text:
        colors.append('tab:blue')
    elif w == target2_text:
        colors.append('tab:green')
    elif w in top1 and w in top2:
        colors.append('tab:purple')  # aparece em ambos
    elif w in top1:
        colors.append('#4da6ff')  # azul claro
    elif w in top2:
        colors.append('#66cc66')  # verde claro
    else:
        colors.append('gray')

origin = np.array([0.0, 0.0])
plt.figure(figsize=(10,6))
for i, w in enumerate(words):
    x, y = proj[i]
    width_head = 0.06 * max(1, np.linalg.norm(proj[i]))
    length_head = 0.09 * max(1, np.linalg.norm(proj[i]))
    linewidth = 2.2 if w in (target1_text, target2_text) else 1.2
    plt.arrow(origin[0], origin[1], x, y,
              head_width=width_head, head_length=length_head,
              fc=colors[i], ec=colors[i], length_includes_head=True,
              alpha=0.9, linewidth=linewidth)
    plt.text(x*1.06, y*1.06, w, fontsize=12, ha='center', va='center', color=colors[i])

plt.axhline(0, color='gray', linewidth=0.5)
plt.axvline(0, color='gray', linewidth=0.5)
plt.grid(True, linestyle='--', alpha=0.4)
plt.title(f"Vetores de palavras em 2D — '{target1_text}' (azul) e '{target2_text}' (verde)")
plt.xlabel("Dimensão 1")
plt.ylabel("Dimensão 2")
plt.tight_layout()
plt.show()


In [ ]:
# branco (azul) e negro (verde)
target1_text = "branco"
target2_text = "negro"
t1 = nlp(target1_text)[0]
t2 = nlp(target2_text)[0]

if nlp.vocab.vectors_length == 0:
    raise RuntimeError("O modelo atual não tem vetores (nlp.vocab.vectors_length == 0). "
                       "Instale um modelo com vetores: python -m spacy download pt_core_news_md (ou lg).")

topn = 10
top1 = [w for w,_ in top_n_similar_from_vocab(t1.vector, t1.text, nlp.vocab, n=topn)]
top2 = [w for w,_ in top_n_similar_from_vocab(t2.vector, t2.text, nlp.vocab, n=topn)]

# criar lista única preservando ordem (primeiro alvo1 e seus top, depois alvo2 e seus top não repetidos)
words = [target1_text] + top1 + [w for w in [target2_text] + top2 if w not in ([target1_text] + top1)]

# montar matriz de vetores
vecs = [nlp.vocab[w].vector for w in words]
X = np.vstack(vecs)

# projetar em 2D com SVD
Xc = X - X.mean(axis=0)
U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
proj = Xc.dot(Vt[:2].T)

# determinar cor por origem
colors = []
for w in words:
    if w == target1_text:
        colors.append('tab:blue')
    elif w == target2_text:
        colors.append('tab:green')
    elif w in top1 and w in top2:
        colors.append('tab:purple')  # aparece em ambos
    elif w in top1:
        colors.append('#4da6ff')  # azul claro
    elif w in top2:
        colors.append('#66cc66')  # verde claro
    else:
        colors.append('gray')

origin = np.array([0.0, 0.0])
plt.figure(figsize=(10,6))
for i, w in enumerate(words):
    x, y = proj[i]
    width_head = 0.06 * max(1, np.linalg.norm(proj[i]))
    length_head = 0.09 * max(1, np.linalg.norm(proj[i]))
    linewidth = 2.2 if w in (target1_text, target2_text) else 1.2
    plt.arrow(origin[0], origin[1], x, y,
              head_width=width_head, head_length=length_head,
              fc=colors[i], ec=colors[i], length_includes_head=True,
              alpha=0.9, linewidth=linewidth)
    plt.text(x*1.06, y*1.06, w, fontsize=12, ha='center', va='center', color=colors[i])

plt.axhline(0, color='gray', linewidth=0.5)
plt.axvline(0, color='gray', linewidth=0.5)
plt.grid(True, linestyle='--', alpha=0.4)
plt.title(f"Vetores de palavras em 2D — '{target1_text}' (azul) e '{target2_text}' (verde)")
plt.xlabel("Dimensão 1")
plt.ylabel("Dimensão 2")
plt.tight_layout()
plt.show()

In [ ]:
# escravo (azul) e senhor (verde)
target1_text = "escravo"
target2_text = "senhor"

t1 = nlp(target1_text)[0]
t2 = nlp(target2_text)[0]

if nlp.vocab.vectors_length == 0:
    raise RuntimeError("O modelo atual não tem vetores (nlp.vocab.vectors_length == 0). "
                       "Instale um modelo com vetores: python -m spacy download pt_core_news_md (ou lg).")

topn = 10
top1 = [w for w,_ in top_n_similar_from_vocab(t1.vector, t1.text, nlp.vocab, n=topn)]
top2 = [w for w,_ in top_n_similar_from_vocab(t2.vector, t2.text, nlp.vocab, n=topn)]

# criar lista única preservando ordem (primeiro alvo1 e seus top, depois alvo2 e seus top não repetidos)
words = [target1_text] + top1 + [w for w in [target2_text] + top2 if w not in ([target1_text] + top1)]

# montar matriz de vetores
vecs = [nlp.vocab[w].vector for w in words]
X = np.vstack(vecs)

# projetar em 2D com SVD
Xc = X - X.mean(axis=0)
U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
proj = Xc.dot(Vt[:2].T)

# determinar cor por origem
colors = []
for w in words:
    if w == target1_text:
        colors.append('tab:blue')
    elif w == target2_text:
        colors.append('tab:green')
    elif w in top1 and w in top2:
        colors.append('tab:purple')  # aparece em ambos
    elif w in top1:
        colors.append('#4da6ff')  # azul claro
    elif w in top2:
        colors.append('#66cc66')  # verde claro
    else:
        colors.append('gray')

origin = np.array([0.0, 0.0])
plt.figure(figsize=(10,6))
for i, w in enumerate(words):
    x, y = proj[i]
    width_head = 0.06 * max(1, np.linalg.norm(proj[i]))
    length_head = 0.09 * max(1, np.linalg.norm(proj[i]))
    linewidth = 2.2 if w in (target1_text, target2_text) else 1.2
    plt.arrow(origin[0], origin[1], x, y,
              head_width=width_head, head_length=length_head,
              fc=colors[i], ec=colors[i], length_includes_head=True,
              alpha=0.9, linewidth=linewidth)
    plt.text(x*1.06, y*1.06, w, fontsize=12, ha='center', va='center', color=colors[i])

plt.axhline(0, color='gray', linewidth=0.5)
plt.axvline(0, color='gray', linewidth=0.5)
plt.grid(True, linestyle='--', alpha=0.4)
plt.title(f"Vetores de palavras em 2D — '{target1_text}' (azul) e '{target2_text}' (verde)")
plt.xlabel("Dimensão 1")
plt.ylabel("Dimensão 2")
plt.tight_layout()
plt.show()